In [1]:
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
from scipy.stats import norm
import dash_daq as daq
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots
from dash_bootstrap_templates import load_figure_template

import dash
from dash import Dash, dcc, html, Input, Output, State
import os
import pickle




##################################################



import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model
import numpy as np 
import numpy as np 
import os
import pandas as pd

import re
from nltk import tokenize
import keras.utils as ku
from sklearn.model_selection import train_test_split
import pickle

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
import time

from dash.long_callback import DiskcacheLongCallbackManager
import diskcache
cache = diskcache.Cache("/Users/janoschkorell/Desktop/Wissenschaft/Statistik/Python/Python Projekte/Predict Text from Parties/filesG/")
long_callback_manager = DiskcacheLongCallbackManager(cache)

import datetime
from tkinter import *
from keras.models import load_model

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/janoschkorell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
os.chdir('/Users/janoschkorell/Desktop/Wissenschaft/Statistik/Python/Python Projekte/Predict Text from Parties/filesG/')

In [3]:
#Import models of parties

file = open("FDP1.pkl",'rb')
model_FDP = pickle.load(file)
file.close()

file = open("tokenizer_FDP",'rb')
tokenizer_FDP = pickle.load(file)
file.close()

file = open("max_sequence_len_FDP",'rb')
max_sequence_len_FDP = pickle.load(file)
file.close()

#########################################################

file = open("Grüne1.pkl",'rb')
model_Grüne = pickle.load(file)
file.close()

file = open("tokenizer_Grüne",'rb')
tokenizer_Grüne = pickle.load(file)
file.close()

file = open("max_sequence_len_Grüne",'rb')
max_sequence_len_Grüne = pickle.load(file)
file.close()

#########################################################


model_afd = load_model('afd.h5')

file = open("tokenizer_afd",'rb')
tokenizer_afd = pickle.load(file)
file.close()

file = open("max_sequence_len_afd",'rb')
max_sequence_len_afd= pickle.load(file)
file.close()

In [4]:
os.chdir('/Users/janoschkorell/Desktop/Wissenschaft/Statistik/Python/Python Projekte/Predict Text from Parties/')

In [5]:

app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])


#Html Block
app.layout = html.Div([

    dbc.Container([

            html.Br(),
            dbc.Row([
                html.H1('Predicting text with text from party speeches'), 
                dcc.Markdown('\n'),
                dcc.Markdown("Here you can generate text in the style of three parties from Germany.\
                These models were trained with:"),
                dcc.Markdown('''
                *  LSTMs  
                *  They achieved an accuracy of about 78% and precision of about 91%
                *  Speeches by party members from the German Bundestag
                *  The data was scraped by the user '''),
                dcc.Markdown('\n'),
                dcc.Markdown('Choose the number of sentences and have fun!'),
                dcc.Markdown('\n'),                      
                
                
                
                html.Br(), html.Br(),

            dcc.Textarea(
                        id='textarea-state-example',
                        value='',
                        placeholder="Type something...",
                        style={'border-radius': '1em', 'width': '70%', 'height': 150}),


                ]
            
            ),  html.Br(),

            html.Div( 
                
                [
                    html.Div( 
                        
                        [

                            html.Button('SUBMIT', id='textarea-state-example-button', n_clicks=0,
                                  
                            )
                        ],
                        style={
                            "display": "inline-block",
                            "width": "30%",
                            'border-radius': '0.5em',
                             "verticalAlign": "top"
                        }
                ),

                html.Div(
                    
                    [

                        daq.NumericInput(id='my-numeric-input-1',  value=0,
                        )

                    ],
                        style={
                            "display": "inline-block",
                            
                        
                            'border-radius': '0.5em'

                            }
                        ),
                    ]
               ), html.Br(), html.Br(),




                dbc.Row([html.H1('Die Grünen'),
                        dcc.Textarea( id='textarea-state-example-output_Grüne',  placeholder = "",
                        style={'border-radius': '1em', 'width': '100%', 'height': 100}, className="class--grüne"),
                        dcc.Interval(id='interval-component', interval=1*1000, n_intervals=0)

                        ]
                       
                       
                ), html.Br(), html.Br(),



                dbc.Row([html.H1('AfD'),
                    dcc.Textarea(id='textarea-state-example-output_afd',  placeholder = "",
                    style={'border-radius': '1em', 'width': '100%', 'whiteSpace': 'pre-line', 'height': 100}),



                        ]
                        
                ), html.Br(), html.Br(),



                dbc.Row([html.H1('FDP'),dcc.Textarea(

                        id='textarea-state-example-output_fdp', placeholder = "",
                        style={'border-radius': '1em', 'width': '100%', 'whiteSpace': 'pre-line', 'height': 100}, loading_state = {}),



                    ]),


        ])


])



#callback for reset with new prompt

@app.callback(
    Output('textarea-state-example-output_Grüne', 'value', allow_duplicate=True),
    Output('textarea-state-example-output_afd', 'value', allow_duplicate=True),
    Output('textarea-state-example-output_fdp', 'value', allow_duplicate=True),
    Input('textarea-state-example-button', 'n_clicks'), prevent_initial_call=True
)



def reset(n_clicks):
    
 
    if n_clicks > 0:
    
        return "","",""






#alternating loading block
@app.callback(
    Output('textarea-state-example-output_Grüne', 'placeholder', allow_duplicate=True),
    Output('textarea-state-example-output_afd', 'placeholder', allow_duplicate=True),
    Output('textarea-state-example-output_fdp', 'placeholder', allow_duplicate=True),
    Input('interval-component', 'n_intervals'),
    Input('textarea-state-example-button', 'n_clicks'), prevent_initial_call=True
)



def placeholder(n_intervals, n_clicks):

    if n_clicks > 0:
  
        if n_intervals % 2 == 0:
            
            
            return '\N{Full block}', '\N{Full block}','\N{Full block}'

        else:
            return "", "", ""






        

#text input and predicted text output. Counting of sentences.
@app.callback(
    Output('textarea-state-example-output_Grüne', 'value' ),
    Output('textarea-state-example-output_afd', 'value'),
    Output('textarea-state-example-output_fdp', 'value'),
    Input('textarea-state-example-button', 'n_clicks'),
    State('textarea-state-example', 'value'),
    Input('my-numeric-input-1', 'value') #Number of sentences
)




def update_output(n_clicks, value, num_sent):
    
    
    value = value.lower()

    if n_clicks > 0:
        
        
        #####################################################################


            #Grüne block
            seed_text = str(value)
            next_words = 1000

            for _ in range(next_words):
                token_list = tokenizer_FDP.texts_to_sequences([seed_text])[0]
                token_list = pad_sequences([token_list], maxlen=max_sequence_len_FDP-1, padding='pre')
                predicted = np.argmax(model_FDP.predict(token_list), axis=-1)
                output_word = ""
                for word, index in tokenizer_FDP.word_index.items():
                    if index == predicted:
                        output_word = word
                        break
                #Sentence counting
                seed_text += " " + output_word
                if len(seed_text.split('.')) == num_sent +1:
                    break


            text_fdp= seed_text

        
        
            #AfD block
            seed_text = str(value)
            next_words = 1000

            for _ in range(next_words):
                token_list = tokenizer_afd.texts_to_sequences([seed_text])[0]
                token_list = pad_sequences([token_list], maxlen=max_sequence_len_afd-1, padding='pre')
                predicted = np.argmax(model_afd.predict(token_list), axis=-1)
                output_word = ""
                for word, index in tokenizer_afd.word_index.items():
                    if index == predicted:
                        output_word = word
                        break
                #Sentence counting
                seed_text += " " + output_word
                if len(seed_text.split('.')) == num_sent +1:
                    break


            text_afd= seed_text

        
            #FDP block
            seed_text = str(value)
            next_words = 1000

            for _ in range(next_words):
                token_list = tokenizer_Grüne.texts_to_sequences([seed_text])[0]
                token_list = pad_sequences([token_list], maxlen=max_sequence_len_Grüne-1, padding='pre')
                predicted = np.argmax(model_Grüne.predict(token_list), axis=-1)
                output_word = ""
                for word, index in tokenizer_Grüne.word_index.items():
                    if index == predicted:
                        output_word = word
                        break
                #Sentence counting
                seed_text += " " + output_word
                if len(seed_text.split('.')) == num_sent +1:
                    break
                


            text_grüne= seed_text
            
            
            return text_grüne, text_afd, text_fdp
    
    

if __name__ == '__main__':
    app.run_server("127.0.0.1", port="8050", dev_tools_hot_reload=True)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Apr/2023 12:02:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:02:08] "GET /assets/typography.css?m=1682385872.9050307 HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2023 12:02:08] "GET /assets/custom-scripts.js?m=1682367884.5711865 HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2023 12:02:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:02:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:02:08] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 200 -
[2023-04-25 12:02:08,474] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatc

1/1 [==============================] - 3s 3s/step


127.0.0.1 - - [25/Apr/2023 12:02:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:02:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:02:23] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:02:23] "POST /_dash-update-component HTTP/1.1" 200 -


1/1 [==============================] - 3s 3s/step


...

127.0.0.1 - - [25/Apr/2023 12:03:55] "POST /_dash-update-component HTTP/1.1" 200 -


1/1 [==============================] - 1s 1s/step


127.0.0.1 - - [25/Apr/2023 12:03:55] "POST /_dash-update-component HTTP/1.1" 200 -


1/1 [==============================] - 1s 1s/step


127.0.0.1 - - [25/Apr/2023 12:03:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:03:56] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:03:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:03:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:03:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:03:59] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:03:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:04:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:04:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:04:02] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:04:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:04:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 12:04:04] 